In [1]:
import sys
sys.path[0] = ('/home/labs/waic/omrik/DNN-Challenge')

In [2]:
from fastai.vision import *
from datasets import GlucoseData
from models import Linear
from wis_dnn_challenge import Predictor

root = Path('../data').resolve()
train = root / 'train'
val = root / 'val'

In [6]:
import pre

def get_data(data_dir):
    cgm, meals = pre.get_dfs(data_dir)
    cgm, y = pre.build_cgm(cgm)
    return cgm, y


In [7]:
cgm_t, y_t = get_data(train)
cgm_t.head()

/home/labs/waic/omrik/DNN-Challenge/pre.py:73: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  X = X.loc[y.index].dropna(how='any', axis=0)


GlucoseValue  GlucoseValue -15min  \
id Date                                                     
46 2015-01-05 22:18:00      5.464080             5.099808   
   2015-01-05 22:33:00      5.828352             5.464080   
   2015-01-05 22:48:00      6.101556             5.828352   
   2015-01-05 23:03:00      6.101556             6.101556   
   2015-01-05 23:18:00      6.283692             6.101556   

                        GlucoseValue -30min  GlucoseValue -45min  \
id Date                                                            
46 2015-01-05 22:18:00             5.281944             5.828352   
   2015-01-05 22:33:00             5.099808             5.281944   
   2015-01-05 22:48:00             5.464080             5.099808   
   2015-01-05 23:03:00             5.828352             5.464080   
   2015-01-05 23:18:00             6.101556             5.828352   

                        GlucoseValue -60min  GlucoseValue -75min  \
id Date                                                            
46 2015-01-05 22:18:00             6.192624             6.465828   
   2015-01-05 22:33:00             5.828352             6.192624   
   2015-01-05 22:48:00             5.281944             5.828352   
   2015-01-05 23:03:00             5.099808             5.281944   
   2015-01-05 23:18:00             5.464080             5.099808   

                        GlucoseValue -90min  GlucoseValue -105min  \
id Date                                                             
46 2015-01-05 22:18:00             6.192624              5.646216   
   2015-01-05 22:33:00             6.465828              6.192624   
   2015-01-05 22:48:00             6.192624              6.465828   
   2015-01-05 23:03:00             5.828352              6.192624   
   2015-01-05 23:18:00             5.281944              5.828352   

                        GlucoseValue -120min  GlucoseValue -135min  ...  \
id Date                                                             ...   
46 2015-01-05 22:18:00              5.555148              5.828352  ...   
   2015-01-05 22:33:00              5.646216              5.555148  ...   
   2015-01-05 22:48:00              6.192624              5.646216  ...   
   2015-01-05 23:03:00              6.465828              6.192624  ...   
   2015-01-05 23:18:00              6.192624              6.465828  ...   

                        GlucoseValue -585min  GlucoseValue -600min  \
id Date                                                              
46 2015-01-05 22:18:00              9.744277              6.647964   
   2015-01-05 22:33:00             11.656705              9.744277   
   2015-01-05 22:48:00             11.929909             11.656705   
   2015-01-05 23:03:00             11.292433             11.929909   
   2015-01-05 23:18:00             10.381753             11.292433   

                        GlucoseValue -615min  GlucoseValue -630min  \
id Date                                                              
46 2015-01-05 22:18:00              5.646216              6.010488   
   2015-01-05 22:33:00              6.647964              5.646216   
   2015-01-05 22:48:00              9.744277              6.647964   
   2015-01-05 23:03:00             11.656705              9.744277   
   2015-01-05 23:18:00             11.929909             11.656705   

                        GlucoseValue -645min  GlucoseValue -660min  \
id Date                                                              
46 2015-01-05 22:18:00              6.283692              6.192624   
   2015-01-05 22:33:00              6.010488              6.283692   
   2015-01-05 22:48:00              5.646216              6.010488   
   2015-01-05 23:03:00              6.647964              5.646216   
   2015-01-05 23:18:00              9.744277              6.647964   

                        GlucoseValue -675min  GlucoseValue -690min  \
id Date                                                              
46 2015-01-05 22:18

In [9]:
mean  = cgm_t['GlucoseValue'].mean()
mean

4.704217484033216

In [23]:
class ZeroModel(Module):
        
    def forward(self, x):
        return x.new_zeros((x.shape[0], 8))


In [11]:
class GlucoseData(Dataset):
    def __init__(self, cgm, y):
        self.c = 8
        self.loss_func = nn.MSELoss()
        self.cgm = torch.tensor(cgm.values, dtype=torch.float)
        self.y  = torch.tensor(y.values, dtype=torch.float)
        
    def __len__(self):
        return len(self.cgm)
    
    def __getitem__(self, i):
        return self.cgm[i][None, ], self.y[i]

In [12]:
cgm_val, y_val = get_data(val)

In [14]:
train_ds = GlucoseData(cgm_t, y_t)
val_ds = GlucoseData(cgm_val, y_val)

In [17]:
data = DataBunch.create(train_ds, val_ds, bs=512)

In [24]:
learner = Learner(data, ZeroModel(), loss_func=nn.MSELoss(), metrics=mean_absolute_error)

In [25]:
learner.validate()

[0.69046277, tensor(0.5458)]